<a href="https://colab.research.google.com/github/sweonurulu/colab_notebooks/blob/main/muzayede2_ysa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import numpy as np
import re
import datetime
import string
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import word_tokenize
from snowballstemmer import TurkishStemmer
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from PIL import Image
from io import BytesIO
import tensorflow as tf
import os


#pd.set_option('display.max_colwidth', None)
#pd.reset_option('display.max_colwidth')

## GET DATASET

In [ ]:
# son kaç günün verisini istiyorsun?
no_of_day = 7

In [ ]:
# Verileri al
response = requests.get("http://95.70.184.79:3066/data/get_item_data/{0}".format(no_of_day))

In [ ]:
# Yanıtın JSON verisini al
data = response.json()

In [ ]:
# JSON verisinden item_data ve auction_data'yı DataFrame'lere dönüştür
item_data = pd.DataFrame(data["item_data"])
auction_data = pd.DataFrame(data["auction_data"])

In [ ]:
item_df = item_data.copy()
auction_df = auction_data.copy()

In [ ]:
dataset=pd.merge(left=auction_df,right=item_df,how="outer",on="auction_id")

In [ ]:
df=dataset.copy()

In [ ]:
# Değiştirme işlemi yapılmayacak sütunlar listesi
excluded_columns = ["item_description", "item_subdescription"]

In [ ]:
# Belirtilen sütunlar hariç tüm sütunlarda boş string ve tek boşlukları np.nan ile değiştir
df.loc[:, ~df.columns.isin(excluded_columns)] = df.loc[:, ~df.columns.isin(excluded_columns)].replace({"": np.nan, " ": np.nan})

## EDA

In [ ]:
df.head()

,auction_id,auction_account,auction_description,auction_datetime,auction_url,item_id,item_name,item_lot_no,item_img,item_description,item_reserve_price,item_sold,item_tracking_no,item_bid_no,item_price,item_subdescription,estimate_price
0,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19T20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239741.0,IŞIL ÖZIŞIK,1.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,IŞIL ÖZIŞIK,6.500 TL,0.0,17.0,0.0,NaN,İstanbul - Galata Kulesi\nKağıt Üzeri Sulu Boy...,NaN
1,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19T20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239742.0,Behruz Kuul,2.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,BEHRUZ KUUL,7.000 TL,0.0,13.0,0.0,NaN,"Evler Serisi\nTuval Üzeri Yağlı Boya\nİmzalı, ...",NaN
2,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19T20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239743.0,FİKRET ÖZTÜRK,3.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,FİKRET ÖZTÜRK,5.000 TL,0.0,6.0,0.0,NaN,"Ana Grup\nTuval Üzeri Yağlı Boya\nİmzalı, 2024...",NaN
3,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19T20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239744.0,HAKAN ÖZDİL,4.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,HAKAN ÖZDİL,5.000 TL,0.0,6.0,0.0,NaN,"Kurt \nTuval Üzeri Akrilik\nİmzalı , 2024\nEn ...",NaN
4,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19T20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239745.0,MEHMET ENGÜL,5.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,MEHMET ENGÜL,5.000 TL,0.0,4.0,0.0,NaN,"Ortaköy\nTuval Üzeri Yağlı Boya\nİmzalı, 2024\...",NaN


In [ ]:
df.shape

(14992, 17)

In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
auction_id,14992.0,1.002277e+08,1.053667e+08,4023974.0,7.023950e+06,6.002417e+07,1.610239e+08,3.560241e+08
item_id,14986.0,1.951725e+11,8.871274e+11,40239741.0,4.024113e+09,2.402402e+10,9.502406e+10,6.302384e+12
item_lot_no,14986.0,6.151572e+02,2.679446e+03,1.0,7.600000e+01,1.760000e+02,3.500000e+02,1.930500e+04
item_sold,14986.0,5.599893e-01,4.964048e-01,0.0,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
item_bid_no,14986.0,3.481049e+00,5.751668e+00,0.0,0.000000e+00,1.000000e+00,5.000000e+00,6.900000e+01


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14992 entries, 0 to 14991
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   auction_id           14992 non-null  int64  
 1   auction_account      14992 non-null  object 
 2   auction_description  14992 non-null  object 
 3   auction_datetime     14992 non-null  object 
 4   auction_url          14992 non-null  object 
 5   item_id              14986 non-null  float64
 6   item_name            14986 non-null  object 
 7   item_lot_no          14986 non-null  float64
 8   item_img             14986 non-null  object 
 9   item_description     14986 non-null  object 
 10  item_reserve_price   14986 non-null  object 
 11  item_sold            14986 non-null  float64
 12  item_tracking_no     11711 non-null  object 
 13  item_bid_no          14986 non-null  float64
 14  item_price           8392 non-null   object 
 15  item_subdescription  14986 non-null 

In [ ]:
df.isnull().sum()

auction_id                 0
auction_account            0
auction_description        0
auction_datetime           0
auction_url                0
item_id                    6
item_name                  6
item_lot_no                6
item_img                   6
item_description           6
item_reserve_price         6
item_sold                  6
item_tracking_no        3281
item_bid_no                6
item_price              6600
item_subdescription        6
estimate_price         14689
dtype: int64

In [ ]:
df.nunique()

auction_id                59
auction_account           43
auction_description       58
auction_datetime          32
auction_url               59
item_id                14986
item_name                780
item_lot_no             1329
item_img               14986
item_description       11346
item_reserve_price       291
item_sold                  2
item_tracking_no          62
item_bid_no               47
item_price               387
item_subdescription     8176
estimate_price           100
dtype: int64

In [ ]:
df.dropna(subset=["item_id"], inplace=True)
df["item_id"] = df["item_id"].astype("int64")

In [ ]:
# auction_datetime sütununu datetime türüne dönüştürme
df["auction_datetime"] = pd.to_datetime(df["auction_datetime"])

In [ ]:
df.head()

,auction_id,auction_account,auction_description,auction_datetime,auction_url,item_id,item_name,item_lot_no,item_img,item_description,item_reserve_price,item_sold,item_tracking_no,item_bid_no,item_price,item_subdescription,estimate_price
0,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19 20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239741,IŞIL ÖZIŞIK,1.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,IŞIL ÖZIŞIK,6.500 TL,0.0,17.0,0.0,NaN,İstanbul - Galata Kulesi\nKağıt Üzeri Sulu Boy...,NaN
1,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19 20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239742,Behruz Kuul,2.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,BEHRUZ KUUL,7.000 TL,0.0,13.0,0.0,NaN,"Evler Serisi\nTuval Üzeri Yağlı Boya\nİmzalı, ...",NaN
2,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19 20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239743,FİKRET ÖZTÜRK,3.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,FİKRET ÖZTÜRK,5.000 TL,0.0,6.0,0.0,NaN,"Ana Grup\nTuval Üzeri Yağlı Boya\nİmzalı, 2024...",NaN
3,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19 20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239744,HAKAN ÖZDİL,4.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,HAKAN ÖZDİL,5.000 TL,0.0,6.0,0.0,NaN,"Kurt \nTuval Üzeri Akrilik\nİmzalı , 2024\nEn ...",NaN
4,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19 20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239745,MEHMET ENGÜL,5.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,MEHMET ENGÜL,5.000 TL,0.0,4.0,0.0,NaN,"Ortaköy\nTuval Üzeri Yağlı Boya\nİmzalı, 2024\...",NaN


In [ ]:
# Adım 3: Klasör oluşturma
output_folder = "/content/drive/MyDrive/mzyd/mzyd_img"
os.makedirs(output_folder, exist_ok=True)

In [ ]:
# Resimleri indirme ve dinamik olarak klasör oluşturup kaydetme fonksiyonu
def download_and_save_image(auction_id, item_id, url, auction_datetime, base_folder):
    # auction_datetime formatını kullanarak yıl_ay_gün şeklinde klasör ismi oluşturma
    date_str = auction_datetime.strftime("%Y_%m_%d")
    output_folder = os.path.join(base_folder, date_str, str(auction_id))  # Alt dizin için auction_id kullanımı
    os.makedirs(output_folder, exist_ok=True)

    response = requests.get(url)
    if response.status_code == 200:
        filename = f"{item_id}.jpg"
        filepath = os.path.join(output_folder, filename)

        with open(filepath, "wb") as file:
            file.write(response.content)

        print(f"{filename} adlı dosya {output_folder} yoluna kaydedildi")
    else:
        print(f"{url} adresinden resim indirme başarısız")

In [ ]:
# Adım 4: DataFrame üzerinden yineleme yaparak her URL'yi işleme
"""base_folder = "/content/drive/MyDrive/mzyd/mzyd_img"
for index, row in df.iterrows():
    download_and_save_image(row['item_id'], row['item_img'], row['auction_datetime'], base_folder)"""

'base_folder = "/content/drive/MyDrive/mzyd/mzyd_img"\nfor index, row in df.iterrows():\n    download_and_save_image(row[\'item_id\'], row[\'item_img\'], row[\'auction_datetime\'], base_folder)'

## CATEGORIZATION

 ### item_name' a göre kategorilendirme

In [ ]:
df.dropna(subset=['item_id'], inplace=True)

In [ ]:
df[df["item_id"].isnull()]

,auction_id,auction_account,auction_description,auction_datetime,auction_url,item_id,item_name,item_lot_no,item_img,item_description,item_reserve_price,item_sold,item_tracking_no,item_bid_no,item_price,item_subdescription,estimate_price


In [ ]:
# Müzayede açıklamaları için Kategorileri belirleyen kelimeler ve bu kelimelere karşılık gelen kategoriler
categories_item_name = {
    'obje': 'obje',

    'efemera' : 'efemera',
    'kartpostal' : 'efemera',
    'pul' : 'efemera',
    'mnh' : 'efemera',
    'fdc' : 'efemera',
    'filateli' : "efemera",
    'fi̇lateli̇' : "efemera",

    'tablo' : 'eser',
    'resim' : 'eser',
    'heykel' : 'eser',

    'para': 'nümismatik',
    'banknot': 'nümismatik',

    'kitap' : 'kitap',
    'dergi' : 'kitap',
    'çizgi roman' : 'kitap'
}

In [ ]:
def categorize_item_name(description):
    description = description.lower()  # Küçük harfe dönüştürme
    for keyword, category in categories_item_name.items():
        if keyword in description:
            return category
        else:
            return np.nan

In [ ]:
df['item_category'] = df['item_name'].apply(categorize_item_name)

In [ ]:
df.head()

,auction_id,auction_account,auction_description,auction_datetime,auction_url,item_id,item_name,item_lot_no,item_img,item_description,item_reserve_price,item_sold,item_tracking_no,item_bid_no,item_price,item_subdescription,estimate_price,item_category
0,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19 20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239741,IŞIL ÖZIŞIK,1.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,IŞIL ÖZIŞIK,6.500 TL,0.0,17.0,0.0,NaN,İstanbul - Galata Kulesi\nKağıt Üzeri Sulu Boy...,NaN,NaN
1,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19 20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239742,Behruz Kuul,2.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,BEHRUZ KUUL,7.000 TL,0.0,13.0,0.0,NaN,"Evler Serisi\nTuval Üzeri Yağlı Boya\nİmzalı, ...",NaN,NaN
2,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19 20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239743,FİKRET ÖZTÜRK,3.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,FİKRET ÖZTÜRK,5.000 TL,0.0,6.0,0.0,NaN,"Ana Grup\nTuval Üzeri Yağlı Boya\nİmzalı, 2024...",NaN,NaN
3,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19 20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239744,HAKAN ÖZDİL,4.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,HAKAN ÖZDİL,5.000 TL,0.0,6.0,0.0,NaN,"Kurt \nTuval Üzeri Akrilik\nİmzalı , 2024\nEn ...",NaN,NaN
4,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19 20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239745,MEHMET ENGÜL,5.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,MEHMET ENGÜL,5.000 TL,0.0,4.0,0.0,NaN,"Ortaköy\nTuval Üzeri Yağlı Boya\nİmzalı, 2024\...",NaN,NaN


In [ ]:
df.isnull().sum()

auction_id                 0
auction_account            0
auction_description        0
auction_datetime           0
auction_url                0
item_id                    0
item_name                  0
item_lot_no                0
item_img                   0
item_description           0
item_reserve_price         0
item_sold                  0
item_tracking_no        3275
item_bid_no                0
item_price              6594
item_subdescription        0
estimate_price         14683
item_category          13643
dtype: int64

 ### auction_description'a göre kategorilendirme

In [ ]:
# Müzayede açıklamaları için Kategorileri belirleyen kelimeler ve bu kelimelere karşılık gelen kategoriler
categories = {
    'para': 'nümismatik',
    'banknot': 'nümismatik',
    'nümismatik': 'nümismatik',
    'nümi̇smati̇k' : "nümismatik",
    'numismatik' : 'nümismatik',
    'numi̇smati̇k' : 'nümismatik',
    'nümizmatik' : 'nümismatik',
    'kağit' : 'nümismatik',

    'fdc' : 'efemera',
    'pul' : 'efemera',
    'kartpostal' : 'efemera',
    'filateli' : "efemera",
    'fi̇lateli̇' : "efemera",
    'efemera' : 'efemera',
    'fila' : 'efemera',

    'obje': 'obje',

    'resim' : 'eser',
    'heykel' : 'eser',
    'tablo' : 'eser',
    'eser' : 'eser',
    'sanat' : 'eser',
    'resi̇m' : 'eser',
    'antika' : 'eser',
    'art' : 'eser',

    'kitap' : 'kitap',
    'kitabhane' : 'kitap',
    'çi̇zgi̇ roman' : 'kitap',
    'çizgi roman' : 'kitap',
    'çi̇zgi roman' : 'kitap',
    'çizgi roman' : 'kitap',
    'dergi' : 'kitap',
    'ki̇tap' : 'kitap'
}

In [ ]:
def categorize_auctions(description):
    description = description.lower()  # Küçük harfe dönüştürme
    for keyword, category in categories.items():
        if keyword in description:
            return category
    return 'diğer'

In [ ]:
df['item_category'].fillna(df['auction_description'].apply(categorize_auctions), inplace=True)

In [ ]:
df["item_category"].value_counts()

item_category
nümismatik    3730
efemera       3543
diğer         2962
eser          2957
obje          1406
kitap          388
Name: count, dtype: int64

In [ ]:
pd.crosstab(df["item_category"], df["item_sold"])

item_sold,0.0,1.0
item_category,,
diğer,1371,1591
efemera,2007,1536
eser,1457,1500
kitap,56,332
nümismatik,1005,2725
obje,698,708


In [ ]:
df.head()

,auction_id,auction_account,auction_description,auction_datetime,auction_url,item_id,item_name,item_lot_no,item_img,item_description,item_reserve_price,item_sold,item_tracking_no,item_bid_no,item_price,item_subdescription,estimate_price,item_category
0,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19 20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239741,IŞIL ÖZIŞIK,1.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,IŞIL ÖZIŞIK,6.500 TL,0.0,17.0,0.0,NaN,İstanbul - Galata Kulesi\nKağıt Üzeri Sulu Boy...,NaN,eser
1,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19 20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239742,Behruz Kuul,2.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,BEHRUZ KUUL,7.000 TL,0.0,13.0,0.0,NaN,"Evler Serisi\nTuval Üzeri Yağlı Boya\nİmzalı, ...",NaN,eser
2,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19 20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239743,FİKRET ÖZTÜRK,3.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,FİKRET ÖZTÜRK,5.000 TL,0.0,6.0,0.0,NaN,"Ana Grup\nTuval Üzeri Yağlı Boya\nİmzalı, 2024...",NaN,eser
3,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19 20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239744,HAKAN ÖZDİL,4.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,HAKAN ÖZDİL,5.000 TL,0.0,6.0,0.0,NaN,"Kurt \nTuval Üzeri Akrilik\nİmzalı , 2024\nEn ...",NaN,eser
4,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19 20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239745,MEHMET ENGÜL,5.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,MEHMET ENGÜL,5.000 TL,0.0,4.0,0.0,NaN,"Ortaköy\nTuval Üzeri Yağlı Boya\nİmzalı, 2024\...",NaN,eser


In [ ]:
df.isnull().sum()

auction_id                 0
auction_account            0
auction_description        0
auction_datetime           0
auction_url                0
item_id                    0
item_name                  0
item_lot_no                0
item_img                   0
item_description           0
item_reserve_price         0
item_sold                  0
item_tracking_no        3275
item_bid_no                0
item_price              6594
item_subdescription        0
estimate_price         14683
item_category              0
dtype: int64

### CATEGORY TO INT

In [ ]:
dummy_cols = pd.get_dummies(df["item_category"], drop_first=True)

In [ ]:
df = pd.concat([df, dummy_cols], axis=1)

In [ ]:
df.drop("item_category",axis=1,inplace=True)

In [ ]:
df.head()

,auction_id,auction_account,auction_description,auction_datetime,auction_url,item_id,item_name,item_lot_no,item_img,item_description,...,item_tracking_no,item_bid_no,item_price,item_subdescription,estimate_price,efemera,eser,kitap,nümismatik,obje
0,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19 20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239741,IŞIL ÖZIŞIK,1.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,IŞIL ÖZIŞIK,...,17.0,0.0,NaN,İstanbul - Galata Kulesi\nKağıt Üzeri Sulu Boy...,NaN,False,True,False,False,False
1,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19 20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239742,Behruz Kuul,2.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,BEHRUZ KUUL,...,13.0,0.0,NaN,"Evler Serisi\nTuval Üzeri Yağlı Boya\nİmzalı, ...",NaN,False,True,False,False,False
2,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19 20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239743,FİKRET ÖZTÜRK,3.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,FİKRET ÖZTÜRK,...,6.0,0.0,NaN,"Ana Grup\nTuval Üzeri Yağlı Boya\nİmzalı, 2024...",NaN,False,True,False,False,False
3,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19 20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239744,HAKAN ÖZDİL,4.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,HAKAN ÖZDİL,...,6.0,0.0,NaN,"Kurt \nTuval Üzeri Akrilik\nİmzalı , 2024\nEn ...",NaN,False,True,False,False,False
4,4023974,Janus Mezat,Janus Resim Müzayedesi,2024-05-19 20:02:00,https://www.muzayedeapp.com/muzayede/janus-res...,40239745,MEHMET ENGÜL,5.0,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,MEHMET ENGÜL,...,4.0,0.0,NaN,"Ortaköy\nTuval Üzeri Yağlı Boya\nİmzalı, 2024\...",NaN,False,True,False,False,False


## PREPROCESSING

**takip sayısı sütunundaki boş değerleri 0 ile dolduralım.**

In [ ]:
df["item_tracking_no"] = df["item_tracking_no"].replace(np.nan,0)

**veritiplerini düzenleyelim.**

In [ ]:
df["auction_account"]=df["auction_account"].astype("string")
df["auction_datetime"] = pd.to_datetime(df["auction_datetime"])
df["auction_description"]=df["auction_description"].astype("string")
df["item_name"]=df["item_name"].astype("string")
df["item_img"]=df["item_img"].astype("string")
df["item_description"]=df["item_description"].astype("string")
df["item_bid_no"]=df["item_bid_no"].astype("uint")
df["item_lot_no"]=df["item_lot_no"].astype("uint")
df["item_reserve_price"]=df["item_reserve_price"].astype("string")
df["item_sold"]=df["item_sold"].astype("int16")
df["item_tracking_no"]=df["item_tracking_no"].astype("uint")
df["item_subdescription"]=df["item_subdescription"].astype("string")

**Tarih sütununu parçalara ayıralım.**

In [ ]:
df[["auction_datetime_day", "auction_datetime_month", "auction_datetime_year",
    "auction_datetime_hour", "auction_datetime_minute"]] = df["auction_datetime"].apply(
        lambda dt: pd.Series([dt.day, dt.month, dt.year, dt.hour, dt.minute]))

**USD verileri TL birimine dönüştürelim.**

In [ ]:
df = df[df["item_reserve_price"].astype(str).str.endswith("TL")]

**Açılış fiyatı kısmındaki TL ibaresini silelim ve Açılış fiyatı verilerini integer formate çevirelim.**

In [ ]:
df["item_reserve_price"] = df["item_reserve_price"].str.replace(".", "").str.replace(" TL", "").astype(int)

**İhtiyaç duymayacağımız sütunları silelim.**

In [ ]:
df.drop(["auction_url","auction_id","item_id","estimate_price","item_price","auction_datetime","auction_account","item_name"],axis=1,inplace=True)

**İki DataFrame'deki açıklama ve alt açıklama sütunlarını birleştirerek tek bir sütun oluşturalım.**

In [ ]:
df["item_merged_description"] = df['item_description'].str.cat(df['item_subdescription'], sep=' ')

In [ ]:
df["item_merged_description"] = df['item_merged_description'].str.cat(df['auction_description'], sep=' ')

**açıklama ve alt açıklama sütunları ile işimiz kalmadı. onları silelim.**

In [ ]:
df.drop(["item_description","item_subdescription","auction_description"],axis=1,inplace=True)

In [ ]:
df.head()

,item_lot_no,item_img,item_reserve_price,item_sold,item_tracking_no,item_bid_no,efemera,eser,kitap,nümismatik,obje,auction_datetime_day,auction_datetime_month,auction_datetime_year,auction_datetime_hour,auction_datetime_minute,item_merged_description
0,1,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,6500,0,17,0,False,True,False,False,False,19,5,2024,20,2,IŞIL ÖZIŞIK İstanbul - Galata Kulesi Kağıt Üz...
1,2,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,7000,0,13,0,False,True,False,False,False,19,5,2024,20,2,BEHRUZ KUUL Evler Serisi Tuval Üzeri Yağlı Bo...
2,3,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,5000,0,6,0,False,True,False,False,False,19,5,2024,20,2,FİKRET ÖZTÜRK Ana Grup Tuval Üzeri Yağlı Boya...
3,4,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,5000,0,6,0,False,True,False,False,False,19,5,2024,20,2,HAKAN ÖZDİL Kurt Tuval Üzeri Akrilik İmzalı ...
4,5,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,5000,0,4,0,False,True,False,False,False,19,5,2024,20,2,MEHMET ENGÜL Ortaköy Tuval Üzeri Yağlı Boya İ...


## NLP

In [ ]:
str_cols =["item_merged_description"]

**Tüm metinsel verileri küçük harfe dönüştürelim.**

In [ ]:
for col in str_cols:
  df[col] = df[col].str.lower()

**Sütunların içindeki noktalama işaretlerini silme işlemi**

In [ ]:
punctutations = string.punctuation

In [ ]:
for col in str_cols:
    df[col] = df[col].apply(lambda x: ''.join(char if char not in punctutations else ' ' for char in x))

**Türkçe stopwordsleri indirelim.**

In [ ]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
sw = stopwords.words("turkish")

**Stopwordsleri silelim.**

In [ ]:
for col in str_cols:
  df[col] = df[col].apply(lambda x: " ".join(x for x in str(x).split() if x not in sw))

**Tokenization**

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
for col in str_cols:
  df[col] = df[col].apply(word_tokenize)

**Stemming**

In [ ]:
stemmer=TurkishStemmer()

In [ ]:
for col in str_cols:
  df[col] = df[col].apply(lambda x: [stemmer.stemWord(word) for word in x])

**Vectorization - Word2Vec**

In [ ]:
# Word2Vec modelini eğitme
#word2vec_model_auction = Word2Vec(df['auction_description'], vector_size=100, window=5, min_count=1, workers=4)
word2vec_model_item = Word2Vec(df['item_merged_description'], vector_size=100, window=5, min_count=1, workers=4)

# Her kelimenin vektörünü almak için bir fonksiyon tanımlama
def get_word_vector(model, word):
    try:
        return model.wv[word]
    except KeyError:
        # Eğer kelime modelde bulunmuyorsa rastgele bir vektör döndür
        return [0] * model.vector_size

# Her kelimenin vektörünü içeren bir sütun oluşturma
#df['auction_description_word2vec'] = df['auction_description'].apply(lambda x: [get_word_vector(word2vec_model_auction, word) for word in x])
df['item_description_word2vec'] = df['item_merged_description'].apply(lambda x: [get_word_vector(word2vec_model_item, word) for word in x])

**vektörize ettiğimiz sütunları veriden silelim.**

In [ ]:
df.drop(["item_merged_description"],axis=1,inplace=True)

In [ ]:
df.head()

,item_lot_no,item_img,item_reserve_price,item_sold,item_tracking_no,item_bid_no,efemera,eser,kitap,nümismatik,obje,auction_datetime_day,auction_datetime_month,auction_datetime_year,auction_datetime_hour,auction_datetime_minute,item_description_word2vec
0,1,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,6500,0,17,0,False,True,False,False,False,19,5,2024,20,2,"[[-0.021024726, -0.023357077, 0.034696314, 0.0..."
1,2,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,7000,0,13,0,False,True,False,False,False,19,5,2024,20,2,"[[-0.034293845, -0.03236682, 0.03248742, 0.041..."
2,3,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,5000,0,6,0,False,True,False,False,False,19,5,2024,20,2,"[[-0.031506393, -0.031377915, 0.01319145, 0.02..."
3,4,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,5000,0,6,0,False,True,False,False,False,19,5,2024,20,2,"[[-0.050853733, 0.067724995, -0.003236102, 0.0..."
4,5,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,5000,0,4,0,False,True,False,False,False,19,5,2024,20,2,"[[-0.3199431, -0.057591323, 0.06582748, 0.3091..."


## IMAGE PROCESSING

In [ ]:
#df["item_category"].unique()

In [ ]:
"""df_eser = df[df["item_category"]=="eser"]
df_obje = df[df["item_category"]=="obje"]
df_numismatik = df[df["item_category"]=="nümismatik"]
df_kitap = df[df["item_category"]=="kitap"]
df_efemera = df[df["item_category"]=="efemera"]
df_diger = df[df["item_category"]=="diğer"]"""

'df_eser = df[df["item_category"]=="eser"]\ndf_obje = df[df["item_category"]=="obje"]\ndf_numismatik = df[df["item_category"]=="nümismatik"]\ndf_kitap = df[df["item_category"]=="kitap"]\ndf_efemera = df[df["item_category"]=="efemera"]\ndf_diger = df[df["item_category"]=="diğer"]'

In [ ]:
"""df_eser.drop(["item_img","item_category"],axis=1,inplace=True)
df_obje.drop(["item_img","item_category"],axis=1,inplace=True)
df_numismatik.drop(["item_img","item_category"],axis=1,inplace=True)
df_kitap.drop(["item_img","item_category"],axis=1,inplace=True)
df_efemera.drop(["item_img","item_category"],axis=1,inplace=True)
df_diger.drop(["item_img","item_category"],axis=1,inplace=True)"""

'df_eser.drop(["item_img","item_category"],axis=1,inplace=True)\ndf_obje.drop(["item_img","item_category"],axis=1,inplace=True)\ndf_numismatik.drop(["item_img","item_category"],axis=1,inplace=True)\ndf_kitap.drop(["item_img","item_category"],axis=1,inplace=True)\ndf_efemera.drop(["item_img","item_category"],axis=1,inplace=True)\ndf_diger.drop(["item_img","item_category"],axis=1,inplace=True)'

In [ ]:
#df_eser = df[df["eser"]==True]

In [ ]:
def download_and_convert_image(url):
  # URL'den görüntüyü indirme
  response = requests.get(url)
  # İndirilen görüntüyü açma
  img = Image.open(BytesIO(response.content))
  # Görüntüyü NumPy dizisine dönüştürme
  img_array = np.array(img)
  return img_array

In [ ]:
# Verileri işlemeden önce görüntülerin boyutunu optimize etme
#df_eser['item_img_data'] = df_eser['item_img'].apply(download_and_convert_image)

In [ ]:
df.head()

,item_lot_no,item_img,item_reserve_price,item_sold,item_tracking_no,item_bid_no,efemera,eser,kitap,nümismatik,obje,auction_datetime_day,auction_datetime_month,auction_datetime_year,auction_datetime_hour,auction_datetime_minute,item_description_word2vec
0,1,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,6500,0,17,0,False,True,False,False,False,19,5,2024,20,2,"[[-0.021024726, -0.023357077, 0.034696314, 0.0..."
1,2,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,7000,0,13,0,False,True,False,False,False,19,5,2024,20,2,"[[-0.034293845, -0.03236682, 0.03248742, 0.041..."
2,3,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,5000,0,6,0,False,True,False,False,False,19,5,2024,20,2,"[[-0.031506393, -0.031377915, 0.01319145, 0.02..."
3,4,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,5000,0,6,0,False,True,False,False,False,19,5,2024,20,2,"[[-0.050853733, 0.067724995, -0.003236102, 0.0..."
4,5,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,5000,0,4,0,False,True,False,False,False,19,5,2024,20,2,"[[-0.3199431, -0.057591323, 0.06582748, 0.3091..."


## YSA

In [ ]:
df.head()

,item_lot_no,item_img,item_reserve_price,item_sold,item_tracking_no,item_bid_no,efemera,eser,kitap,nümismatik,obje,auction_datetime_day,auction_datetime_month,auction_datetime_year,auction_datetime_hour,auction_datetime_minute,item_description_word2vec
0,1,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,6500,0,17,0,False,True,False,False,False,19,5,2024,20,2,"[[-0.021024726, -0.023357077, 0.034696314, 0.0..."
1,2,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,7000,0,13,0,False,True,False,False,False,19,5,2024,20,2,"[[-0.034293845, -0.03236682, 0.03248742, 0.041..."
2,3,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,5000,0,6,0,False,True,False,False,False,19,5,2024,20,2,"[[-0.031506393, -0.031377915, 0.01319145, 0.02..."
3,4,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,5000,0,6,0,False,True,False,False,False,19,5,2024,20,2,"[[-0.050853733, 0.067724995, -0.003236102, 0.0..."
4,5,https://d35fbhjemrkr2a.cloudfront.net/Images/S...,5000,0,4,0,False,True,False,False,False,19,5,2024,20,2,"[[-0.3199431, -0.057591323, 0.06582748, 0.3091..."


In [ ]:
df.drop(["item_img","item_description_word2vec"],axis=1,inplace=True)

In [ ]:
X = df.drop("item_sold",axis=1)
y = df["item_sold"]

In [ ]:
df.head()

,item_lot_no,item_reserve_price,item_sold,item_tracking_no,item_bid_no,efemera,eser,kitap,nümismatik,obje,auction_datetime_day,auction_datetime_month,auction_datetime_year,auction_datetime_hour,auction_datetime_minute
0,1,6500,0,17,0,False,True,False,False,False,19,5,2024,20,2
1,2,7000,0,13,0,False,True,False,False,False,19,5,2024,20,2
2,3,5000,0,6,0,False,True,False,False,False,19,5,2024,20,2
3,4,5000,0,6,0,False,True,False,False,False,19,5,2024,20,2
4,5,5000,0,4,0,False,True,False,False,False,19,5,2024,20,2


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
# Verileri float türüne dönüştürme
X_train = X_train.astype(float)
X_val = X_val.astype(float)
y_train = y_train.astype(float)
y_val = y_val.astype(float)
X_test = X_test.astype(float)
y_test = y_test.astype(float)

In [ ]:
y_train.shape

(9590,)

In [ ]:
# Model oluşturma
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# Model derleme
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # İki sınıflı çapraz entropi kayıp fonksiyonu
              metrics=['accuracy'])

In [ ]:
X_train.head()

,item_lot_no,item_reserve_price,item_tracking_no,item_bid_no,efemera,eser,kitap,nümismatik,obje,auction_datetime_day,auction_datetime_month,auction_datetime_year,auction_datetime_hour,auction_datetime_minute
6341,185.0,20.0,4.0,13.0,1.0,0.0,0.0,0.0,0.0,18.0,5.0,2024.0,13.0,2.0
4346,69.0,1.0,4.0,13.0,0.0,1.0,0.0,0.0,0.0,18.0,5.0,2024.0,21.0,1.0
6989,169.0,100.0,14.0,15.0,0.0,0.0,0.0,0.0,1.0,21.0,5.0,2024.0,21.0,32.0
4832,555.0,1.0,8.0,23.0,0.0,1.0,0.0,0.0,0.0,18.0,5.0,2024.0,21.0,1.0
14383,302.0,25.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,19.0,5.0,2024.0,21.0,32.0


In [ ]:
y_train.head()

6341     1.0
4346     1.0
6989     1.0
4832     1.0
14383    0.0
Name: item_sold, dtype: float64

In [ ]:
X_train.head()

,item_lot_no,item_reserve_price,item_tracking_no,item_bid_no,efemera,eser,kitap,nümismatik,obje,auction_datetime_day,auction_datetime_month,auction_datetime_year,auction_datetime_hour,auction_datetime_minute
6341,185.0,20.0,4.0,13.0,1.0,0.0,0.0,0.0,0.0,18.0,5.0,2024.0,13.0,2.0
4346,69.0,1.0,4.0,13.0,0.0,1.0,0.0,0.0,0.0,18.0,5.0,2024.0,21.0,1.0
6989,169.0,100.0,14.0,15.0,0.0,0.0,0.0,0.0,1.0,21.0,5.0,2024.0,21.0,32.0
4832,555.0,1.0,8.0,23.0,0.0,1.0,0.0,0.0,0.0,18.0,5.0,2024.0,21.0,1.0
14383,302.0,25.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,19.0,5.0,2024.0,21.0,32.0


In [ ]:
# Model eğitimi
model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

Epoch 1/20
300/300 [==============================] - 2s 4ms/step - loss: 17.3483 - accuracy: 0.6321 - val_loss: 12.1577 - val_accuracy: 0.7377
Epoch 2/20
300/300 [==============================] - 1s 3ms/step - loss: 19.3540 - accuracy: 0.7217 - val_loss: 26.4243 - val_accuracy: 0.7473
Epoch 3/20
300/300 [==============================] - 1s 3ms/step - loss: 11.5010 - accuracy: 0.7265 - val_loss: 26.3821 - val_accuracy: 0.6656
Epoch 4/20
300/300 [==============================] - 1s 2ms/step - loss: 17.3611 - accuracy: 0.7542 - val_loss: 55.6206 - val_accuracy: 0.8082
Epoch 5/20
300/300 [==============================] - 1s 2ms/step - loss: 11.8857 - accuracy: 0.7490 - val_loss: 38.8510 - val_accuracy: 0.7877
Epoch 6/20
300/300 [==============================] - 1s 2ms/step - loss: 9.4263 - accuracy: 0.7743 - val_loss: 9.9692 - val_accuracy: 0.8428
Epoch 7/20
300/300 [==============================] - 1s 2ms/step - loss: 13.4734 - accuracy: 0.7914 - val_loss: 5.8014 - val_accuracy: 0.

In [ ]:
model.save("muzayede_model_v1.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from keras.models import load_model

In [ ]:
model_ysa = load_model("muzayede_model_v1.h5")

In [ ]:
# Tek bir veri noktası
predict_data = [132.0, 50.0, 3.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 19.0, 5.0, 2024.0, 20.0, 2.0]
predict_data2 = [215.0,	500.0,	2.0,	11.0,	0.0,	0.0,	0.0,	0.0,	0.0,	19.0,	5.0,	2024.0,	14.0,	1.0]

In [ ]:
# Giriş verisini uygun şekle getirme (1, n_features)
predict_data2 = np.array(predict_data).reshape(1, -1)

In [ ]:
pred = model_ysa.predict(X_test)

94/94 [==============================] - 0s 1ms/step


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
pred = np.argmax(pred, axis=1)

In [ ]:
accuracy_score(y_true=y_test, y_pred=pred)

0.43162108072048033